Test

In [10]:
import numpy as np

assert np.array([1,2,3])@np.array([1,2,3]) == 14, 'Math is not mathing'

In [11]:
print("hallo wereld")

hallo wereld


In [12]:
from mesa import Agent
from mesa import Model

In [269]:
#https://mesa.readthedocs.io/stable/tutorials/intro_tutorial.html

class FED_Agent(Agent):
    """Agent that models the FED (Central bank of US)"""

    def __init__(self, model):
        super().__init__(model)
        self.interest_rate = 4.75 #current

    #test print uit tutorial
    def print_func(self):
        print(f"The interest rate = {str(self.interest_rate)}. ID = {str(self.unique_id)}.")

class Bank_Agent(Agent):
    """Agent that models the banks"""
    def __init__(self, model):
        super().__init__(model)
        self.money_in_banks = 100 #in trillions oid.
        self.profit_precentage = 0 

    def print_money_in_bank(self):
        print(f"Money in Bank is: {self.money_in_banks}")

    def print_profits(self):
        print(f"Profit is: {self.profit_precentage}")

    def change_profits(self, num):
        self.profit_precentage = num

class Energy_Sector_Agent(Agent):
    """Agent that models the energy sector"""
    def __init__(self, model):
        super().__init__(model)
        self.change_in_energy_price = 0

class Real_Estat_Agent(Agent):
    def __init__(self, model, avarege_cost_house = 250000, mortage_rate = 4.75, new_houses = 0, house_buyers = 100):
        super().__init__(model)
        self.avarege_cost_house = avarege_cost_house
        self.mortage_rate = mortage_rate
        self.avarege_cost_mortage = mortage_rate / 100 * avarege_cost_house
        self.new_houses = new_houses
        self.house_buyers = house_buyers

    def print_cost_house(self):
        print(f"cost of house = {self.avarege_cost_house}")

    def step(self, interest_rate):
        #zet stap door alleen te kijken naar interest rate en op basis daarvan hoeveelheid nieuwe huizen gebouwd.
        if interest_rate <= 0:
            self.new_houses = 110 #interest rate van 0 -> goedkoop lenen, dus huizen bouwen met geleend geld is goedkoper dus meer huizen gebouwd dan vraag naar huizen is.
        else:
            self.new_houses = 110 - (interest_rate * 3) # higher interest rate = more costs lending = more costs houses = less houses build
            #interest rate * 3 is op weinig gebasseerd
        shortage_houses = int(self.house_buyers - self.new_houses)
        self.avarege_cost_house += 100 * shortage_houses

        
class Model1(Model):
    """Model for attempt 1."""

    def __init__(self, n, seed=None):
        super().__init__(seed=seed)
        self.num_agents = n

        FED_Agent.create_agents(model = self, n = 1)
        Bank_Agent.create_agents(model = self, n = 1)
        Energy_Sector_Agent.create_agents(model = self, n = 1)
        Real_Estat_Agent.create_agents(model = self, n = 1)

    def house_market_step(self):
        #ga 1 stap verder in model voor alleen house market

        REA = self.agents.select(agent_type = Real_Estat_Agent)
        REA.do("step", 3)
        REA.do("print_cost_house")

        #voor pakken interest rate uit FED

        #FED = self.agents.select(agent_type = FED_Agent)
        #REA.do("step", FED.get("interest_rate")[0])
        #REA.do("print_cost_house")
    
    
    def printing(self):

        #printen profits van banken met verandering in bank door actie change profits
        bank = self.agents.select(agent_type = Bank_Agent)
        fed = self.agents.select(agent_type = FED_Agent)
        print(bank.get("profit_precentage"))
        bank.do("change_profits", fed.get("interest_rate")[0] / 3)
        bank.do("print_profits")

        #printen van alle parameters
        for a in self.agents:
            keys = list(a.__dict__.keys())
            keys = [keys[i] for i in range(len(keys)) if i >=3]
            for key in keys:
                print(key, '=', a.__dict__[key])

In [270]:
#running model
starter_model = Model1(10)
# printen van alle parameters
for _ in range(1):
    starter_model.printing()

[0]
Profit is: 1.5833333333333333
interest_rate = 4.75
money_in_banks = 100
profit_precentage = 1.5833333333333333
change_in_energy_price = 0
avarege_cost_house = 250000
mortage_rate = 4.75
avarege_cost_mortage = 11875.0
new_houses = 0
house_buyers = 100


In [271]:
#10 stappen in house market en printen van kosten van een gemiddeld huis.
for _ in range(10):
    starter_model.house_market_step()

cost of house = 249900
cost of house = 249800
cost of house = 249700
cost of house = 249600
cost of house = 249500
cost of house = 249400
cost of house = 249300
cost of house = 249200
cost of house = 249100
cost of house = 249000


cost of house = 248900
cost of house = 248800
cost of house = 248700
cost of house = 248600
cost of house = 248500
cost of house = 248400
cost of house = 248300
cost of house = 248200
cost of house = 248100
cost of house = 248000
